In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/thol/Downloads/cmpe257_gcloud_jupyter.json"

In [2]:
# Cell 1
from google.cloud import storage
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
from ipywidgets import FloatProgress
from IPython.display import display
import tensorflow as tf
from tensorflow.python.lib.io import file_io

def resize(img):
    shape = img.shape
    if shape[1] > 480 or shape[0] > 320:
        scale = min(480 / shape[1], 320 / shape[0])
        img = cv2.resize(img, dsize=(0,0), fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

    shape = img.shape   
    if shape[1] < 480 or shape[0] < 320:
        v = 320 - shape[0]
        top = v // 2
        bottom = v - top

        h = 480 - shape[1]
        left = h // 2
        right = h - left

        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0, 0))
        
    return img
DIR = "gs://mobgen_training_data/Images/"
out = "gs://mobgen_training_data/train.csv"
imgnms = tf.gfile.ListDirectory(DIR)
imgnms[0] #Should print 'Airi.png'

#Cell 2
f=FloatProgress(min=0, max=703, description='0')
display(f) #Should make a progress bar appear.

#Cell 3
i = 0
while i< len(imgnms):    
    with file_io.FileIO(out, 'wb') as outfile:
        while True: 
            i+=1
            fname = imgnms[i]
            f.value = i
            f.description = str(i)
            with file_io.FileIO(DIR + fname, 'rb') as imgfile:
                img = cv2.imdecode(np.frombuffer(imgfile.read(),np.uint8), cv2.IMREAD_UNCHANGED)
                img = resize(img)

                alpha = img[:,:,3]
                img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

                hl = img[:,:,(0,1)]
                sa = np.dstack([img[:,:,1], alpha])

                hl = np.frompyfunc(lambda acc, nxt: 256 * acc + nxt, 2, 1).reduce(hl, axis=2).ravel() / 256 ** 2
                sa = np.frompyfunc(lambda acc, nxt: 256 * acc + nxt, 2, 1).reduce(sa, axis=2).ravel() / 256 ** 2

                hlsa = np.empty((hl.size + sa.size,), dtype=np.float32)
                hlsa[0::2] = hl
                hlsa[1::2] = sa

                np.savetxt(outfile, hlsa.reshape(1, -1), delimiter=',', header='', footer='')
            if i % 25 == 0:
                break

NotFoundError: Could not find directory